In [62]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
from io import BytesIO
from minio import Minio
from dotenv import load_dotenv

In [66]:
load_dotenv()

url = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/'

minio_client = Minio(
    'minio:9000', 
    access_key=os.getenv('MINIO_KEY_LEITURA_ESCRITA'),
    secret_key=os.getenv('MINIO_SECRET_LEITURA_ESCRITA'),
    secure=False 
)

In [68]:
try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err}")

In [69]:
soup = BeautifulSoup(response.text, 'html.parser')
zip_links = [url + a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.zip')]

In [72]:
for link in zip_links:
    try:
        zip_resp = requests.get(link)
        zip_resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(f"Erro HTTP {link}: {err}")
        
    with zipfile.ZipFile(BytesIO(zip_resp.content)) as zfile:
        for file_name in zfile.namelist():
            with zfile.open(file_name) as file_data:
                minio_client.put_object(
                    'raw',  
                    file_name,    
                    file_data,   
                    length=zfile.getinfo(file_name).file_size 
                )
                print(f"Enviado para Minio: {file_name}")

Enviado para Minio: focos_br_ref_2003.csv
Enviado para Minio: focos_br_ref_2004.csv
Enviado para Minio: focos_br_ref_2005.csv
Enviado para Minio: focos_br_ref_2006.csv
Enviado para Minio: focos_br_ref_2007.csv
Enviado para Minio: focos_br_ref_2008.csv
Enviado para Minio: focos_br_ref_2009.csv
Enviado para Minio: focos_br_ref_2010.csv
Enviado para Minio: focos_br_ref_2011.csv
Enviado para Minio: focos_br_ref_2012.csv
Enviado para Minio: focos_br_ref_2013.csv
Enviado para Minio: focos_br_ref_2014.csv
Enviado para Minio: focos_br_ref_2015.csv
Enviado para Minio: focos_br_ref_2016.csv
Enviado para Minio: focos_br_ref_2017.csv
Enviado para Minio: focos_br_ref_2018.csv
Enviado para Minio: focos_br_ref_2019.csv
Enviado para Minio: focos_br_ref_2020.csv
Enviado para Minio: focos_br_ref_2021.csv
Enviado para Minio: focos_br_ref_2022.csv
Enviado para Minio: focos_br_ref_2023.csv
